# WaterLevel with Deep Sleep v1 


In [1]:
from collections import defaultdict
import json
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture

In [2]:
experiment = "waterlevel2"
power_profiler_freq = 10 #ms
board="nodemcu"
r = 30 #s run time (0 => no deep sleep)
ds = 30 #s deep_sleep time (0 => no deep sleep)
update_interval = 1000 #ms
test_num = 1 #test_number

#Analisys
batteries = [0.8, 2, 4, 10] #Ah
deep_sleep_times = [0, 10, 30, 60, 600, 1800, 3600]

## Code

In [3]:
def load_data(filename, show=False):
    try:
        df = pd.read_csv(filename)
        if show:
            display(df.head())
    except FileNotFoundError:
        print(f"Error: '{fn}' not found.")
    except pd.errors.ParserError:
        print("Error: Unable to parse the CSV file. Check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return df

In [4]:
def preprocess(df): 
    '''
    Check for missing values
    '''
    missing_values = df['current'].isnull().sum()
    missing_percentage = (missing_values / len(df['current'])) * 100
    print(f"Missing values in 'current' column: {missing_values} ({missing_percentage:.2f}%)")

    # Determine data type and convert if necessary
    print(f"Original data type of 'current' column: {df['current'].dtype}")
    if not pd.api.types.is_numeric_dtype(df['current']):
        try:
            df['current'] = pd.to_numeric(df['current'], errors='coerce')
            print("Converted 'current' column to numeric type.")
        except Exception as e:
            print(f"Error converting 'current' column: {e}")
    print(f"Data type of 'current' column after conversion: {df['current'].dtype}")

    # Calculate descriptive statistics
    descriptive_stats = df['current'].describe()
    print("\nDescriptive Statistics for 'current' column:")
    print(descriptive_stats)
    return df

**Reasoning**:
Clean the 'current' column by removing outliers using the IQR method and impute any remaining missing values with the median.


**NOTA: ALGO SE VE RARO EN ESTO, REVISAR EL MÉTODO, PUEDE SER QUE NO ELIMINE NADA **

In [5]:
def clean(df):
    quantile_low = 0.25
    quantile_high = 0.75
    iqr_factor = 1.5

    # Calculate Q1, Q3, and IQR
    Q1 = df['current'].quantile(quantile_low)
    Q3 = df['current'].quantile(quantile_high)
    IQR = Q3 - Q1

    # Define bounds for outlier removal
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"IQR lower_bound: {lower_bound}, upper_bound: {upper_bound})")

    # Remove outliers
    org_len = len(df['current'])
    df = df[~((df['current'] < lower_bound) | (df['current'] > upper_bound))]
    removed_len = len(df['current'])
    print(f"Length before IQR bounds: {org_len}, after: {removed_len}")

    # Display descriptive statistics after outlier removal
    print("\nDescriptive Statistics for 'current' column after outlier removal:")
    print(df['current'].describe())

    # Handle any remaining missing values
    if df['current'].isnull().any():
        median_current = df['current'].median()
        df['current'].fillna(median_current, inplace=True)
        print("\nDescriptive Statistics for 'current' column after imputation:")
        print(df['current'].describe())
    else:
        print("\nNo missing values found after outlier removal.")
    return df

In [6]:
def gmm_fit(data, plot_results=False):
    """
    Finds a current threshold to separate high and low current usage modes using Gaussian Mixture Models.

    Args:
        data (pd.Series or np.array): The current usage data.
        plot_results (bool, optional): Whether to plot the histogram and GMM fit. Defaults to False.

    Returns:
        float: The current threshold separating the modes, or None if the fitting fails.
        array of means
        array of labels
    """

    if isinstance(data, pd.Series):
        data = data.values.reshape(-1, 1) #Reshape for sklearn
    elif isinstance(data, np.ndarray):
        data = data.reshape(-1, 1)
    else:
        raise TypeError("Input data must be a pandas Series or numpy array.")

    try:
        gmm = GaussianMixture(n_components=2)
        labels = gmm.fit_predict(data)
        means = gmm.means_.flatten()
        means.sort()  # Ensure means are sorted: low, high

        # Calculate the threshold as the midpoint between the means
        threshold = (means[0] + means[1]) / 2

        if plot_results:
            plt.hist(data, bins=50, density=True, alpha=0.6, color='g')
            x = np.linspace(data.min(), data.max(), 200).reshape(-1, 1)
            logprob = gmm.score_samples(x)
            pdf = np.exp(logprob)
            plt.plot(x, pdf, '-r')
            plt.axvline(threshold, color='b', linestyle='--', label=f'Threshold: {threshold:.4f}')
            plt.axvline(means[0], color="#888888", linestyle="--", label=f"Mean 0: {means[0]:.4f}")
            plt.axvline(means[1], color="#888888", linestyle="--", label=f"Mean 1: {means[1]:.4f}")
            plt.legend()
            plt.xlabel("Current")
            plt.ylabel("Density")
            plt.title("Current Usage Distribution and GMM Fit")
            plt.show()

        return threshold, means, labels

    except Exception as e:
        print(f"Error during GMM fitting: {e}")
        return None

#Example Usage
# Create a sample dataset (replace with your actual data)

In [7]:
def analyze_cycles(df):
    """
    Analyzes and separates high current usage cycles and deep sleep periods.
    """
    
    cycles = []
    # Iterate over the DataFrame and check for transitions between high and low current states.
    # This is a simplified representation, and the logic may need adjustment based on your specific requirements
    current_state = None
    start_time = None
    for index, row in df.iterrows():
        if row['state_label'] == "high":
            if current_state != "high":  # Transition to high current state
                if current_state == "low":
                    cycles.append({"state":"low", "start_time":start_time, "end_time": row['read_at']})
                current_state = "high"
                start_time = row['read_at']
        else:
            if current_state != "low":  # Transition to low current state
                if current_state == "high":
                    cycles.append({"state":"high", "start_time":start_time, "end_time": row['read_at']})
                current_state = "low"
                start_time = row['read_at']

    # Handle the last cycle
    if start_time:
        cycles.append({"state":current_state, "start_time":start_time, "end_time": df['read_at'].iloc[-1]})

    return cycles

## Analysis

In [8]:
experiment_name = f"{board}_up{update_interval}_r{r}_ds{ds}-t{test_num}"
filename = f"../data/{experiment}-f{power_profiler_freq}--{experiment_name}.csv"
print(experiment_name)
print(filename)
df = load_data(filename, True)

nodemcu_up1000_r30_ds30-t1
../data/waterlevel2-f10--nodemcu_up1000_r30_ds30-t1.csv


index                     read_at  millis            timestamp  voltage  \
0      0  2025-03-12 18:39:30.708691  377438  2025-03-12 18:39:30    5.104   
1      1  2025-03-12 18:39:30.709475  377449  2025-03-12 18:39:30    5.104   
2      2  2025-03-12 18:39:30.710089  377460  2025-03-12 18:39:30    5.104   
3      3  2025-03-12 18:39:30.710703  377471  2025-03-12 18:39:30    5.104   
4      4  2025-03-12 18:39:30.711436  377482  2025-03-12 18:39:30    5.104   

   current    power  bus_voltage  
0   0.0128  0.06533      0.00128  
1   0.0128  0.06533      0.00128  
2   0.0128  0.06533      0.00128  
3   0.0128  0.06533      0.00128  
4   0.0128  0.06533      0.00128

In [9]:
# Convert 'read_at' to datetime objects
df['read_at'] = pd.to_datetime(df['read_at'])

# Calculate descriptive statistics
descriptive_stats = df['current'].describe()
print("Descriptive Statistics for 'current' column:")
print(descriptive_stats)

# Analyze trends using rolling mean
df['rolling_mean'] = df['current'].rolling(window=5, center=False).mean()

threshold, means, labels = gmm_fit(df.current, plot_results=True)

Descriptive Statistics for 'current' column:
count    21021.000000
mean         0.053746
std          0.035626
min          0.012800
25%          0.012800
50%          0.082400
75%          0.083200
max          0.164400
Name: current, dtype: float64


In [10]:
# Visualize trends and anomalies
plt.figure(figsize=(15, 8))
plt.plot(df['read_at'], df['current'], label='Current', color='blue')
plt.plot(df['read_at'], df['rolling_mean'], label='Rolling Mean', color='red')

plt.xlabel('read_at')
plt.ylabel('Current')
plt.title('Current Consumption over Time')
plt.legend()
plt.grid(True)
plt.show()

In [11]:
plt.figure(figsize=(15, 8))
plt.plot(df['read_at'], df['current'], color="gray")
plt.scatter(df['read_at'], df['current'], label='Current', c=labels, cmap=ListedColormap(["blue", "red"]))
#plt.axhline(threshold, color='b', linestyle='--', label=f"threshold={threshold:.4f}")
plt.legend()

In [12]:
states = ["low", "high"]
if means[0] > means[1]:
    states = ["high", "low"]
df['state_label'] = [states[l] for l in labels]

In [13]:
for index, state in zip([0, 1], states):
    print(f"Average current in {state}: {means[index]}")

Average current in low: 0.012800088829668857
Average current in high: 0.08443791926758297


In [14]:

cycles = analyze_cycles(df)

# Example calculations for each cycle
durations = defaultdict(list)
#Excluir la primera y ultima fases capturadas porque estan incompletas
print("State\tStart\tEnd\tDuration")
for cycle in cycles[1:-1]:
    duration = cycle['end_time'] - cycle['start_time']
    durations[cycle['state']].append(duration.total_seconds())
    print(f"{cycle['state']}\t{cycle['start_time']}\t{cycle['end_time']}\t{duration}")
    # Add other calculations as needed (e.g., average current within the cycle, energy consumption)

average_durations = {state: (sum(d) / len(d)) for state, d in durations.items()}
print(f"Average durations: {average_durations}")

State	Start	End	Duration
low	2025-03-12 18:39:51.597107	2025-03-12 18:40:30.930280	0 days 00:00:39.333173
high	2025-03-12 18:40:30.930280	2025-03-12 18:40:58.213438	0 days 00:00:27.283158
low	2025-03-12 18:40:58.213438	2025-03-12 18:41:37.542376	0 days 00:00:39.328938
high	2025-03-12 18:41:37.542376	2025-03-12 18:42:04.862547	0 days 00:00:27.320171
low	2025-03-12 18:42:04.862547	2025-03-12 18:42:44.195622	0 days 00:00:39.333075
high	2025-03-12 18:42:44.195622	2025-03-12 18:43:11.498502	0 days 00:00:27.302880
Average durations: {'low': 39.33172866666667, 'high': 27.302069666666668}


In [15]:
for cycle in cycles[1:-1]:
    # Filter the DataFrame to include only data points within the current cycle
    cycle_data = df[(df['read_at'] >= cycle['start_time']) & (df['read_at'] <= cycle['end_time'])]

    # Calculate the average current for the current cycle
    avg_current = cycle_data['current'].mean()

    # Print the average current for the current cycle
    print(f"Average Current for Cycle ({cycle['state']}): {avg_current:.4f} A")


Average Current for Cycle (low): 0.0845 A
Average Current for Cycle (high): 0.0128 A
Average Current for Cycle (low): 0.0842 A
Average Current for Cycle (high): 0.0128 A
Average Current for Cycle (low): 0.0841 A
Average Current for Cycle (high): 0.0128 A


## Battery predictions

### deep sleep as measured by power_profiler

In [16]:
avg_high_current = means[states.index('high')]
avg_low_current = means[states.index('low')]

batteries = [0.8, 2, 4, 10] #Ah
avg_current = avg_high_current * average_durations['high'] + avg_low_current * average_durations['low']
avg_current /= average_durations['high'] + average_durations['low']
print(f"Average current: {avg_current * 1000:.2f} mA")

for bat_usable_cap in batteries: #Amp hour
    runtime = bat_usable_cap / avg_current
    print(f"At {bat_usable_cap:.02f}Ah:\t{runtime:.01f} hours,\t{runtime/24:.02f} days")

Average current: 42.15 mA
At 0.80Ah:	19.0 hours,	0.79 days
At 2.00Ah:	47.4 hours,	1.98 days
At 4.00Ah:	94.9 hours,	3.95 days
At 10.00Ah:	237.2 hours,	9.88 days


### deep sleep current measured by multimeter

In [17]:
boards = json.loads(open("../data/boards.json").read())
boards

{'nodemcu': {'current': {'deep_sleep': 0.0092}},
 'saola': {'current': {'deep_sleep': 0.0009}},
 'wemosd1': {'current': {'deep_sleep': 6.5e-05}},
 'wemosc3mini': {'current': {'deep_sleep': 0.000418}}}

In [18]:
deep_sleep_measured_current = boards[board]["current"]["deep_sleep"]

avg_current = avg_high_current * average_durations['high'] + deep_sleep_measured_current * average_durations['low']
avg_current /= average_durations['high'] + average_durations['low']
print(f"Average current: {avg_current * 1000:.2f} mA")

for bat_usable_cap in batteries: #Amp hour
    runtime = bat_usable_cap / avg_current
    print(f"At {bat_usable_cap:.02f}Ah:\t{runtime:.01f} hours,\t{runtime/24:.02f} days")

Average current: 40.03 mA
At 0.80Ah:	20.0 hours,	0.83 days
At 2.00Ah:	50.0 hours,	2.08 days
At 4.00Ah:	99.9 hours,	4.16 days
At 10.00Ah:	249.8 hours,	10.41 days


### predicitions on measured deep sleep

In [19]:
run_time = average_durations['high']
for deep_sleep_time in deep_sleep_times:
    avg_current = avg_high_current * run_time + deep_sleep_measured_current * deep_sleep_time
    avg_current /= run_time + deep_sleep_time
    print(f"\n{board} run_time: {run_time:.2f} seg, deep_sleep_time: {deep_sleep_time:.2f} seg")
    print(f"  average current: {avg_current * 1000:.4f} mA")

    for bat_usable_cap in batteries: #Amp hour
        bat_runtime = bat_usable_cap / avg_current
        print(f"  At {bat_usable_cap:.02f}Ah:\t{bat_runtime:.01f} hours,\t{bat_runtime/24:.02f} days")


nodemcu run_time: 27.30 seg, deep_sleep_time: 0.00 seg
  average current: 84.4379 mA
  At 0.80Ah:	9.5 hours,	0.39 days
  At 2.00Ah:	23.7 hours,	0.99 days
  At 4.00Ah:	47.4 hours,	1.97 days
  At 10.00Ah:	118.4 hours,	4.93 days

nodemcu run_time: 27.30 seg, deep_sleep_time: 10.00 seg
  average current: 64.2680 mA
  At 0.80Ah:	12.4 hours,	0.52 days
  At 2.00Ah:	31.1 hours,	1.30 days
  At 4.00Ah:	62.2 hours,	2.59 days
  At 10.00Ah:	155.6 hours,	6.48 days

nodemcu run_time: 27.30 seg, deep_sleep_time: 30.00 seg
  average current: 45.0478 mA
  At 0.80Ah:	17.8 hours,	0.74 days
  At 2.00Ah:	44.4 hours,	1.85 days
  At 4.00Ah:	88.8 hours,	3.70 days
  At 10.00Ah:	222.0 hours,	9.25 days

nodemcu run_time: 27.30 seg, deep_sleep_time: 60.00 seg
  average current: 32.7292 mA
  At 0.80Ah:	24.4 hours,	1.02 days
  At 2.00Ah:	61.1 hours,	2.55 days
  At 4.00Ah:	122.2 hours,	5.09 days
  At 10.00Ah:	305.5 hours,	12.73 days

nodemcu run_time: 27.30 seg, deep_sleep_time: 600.00 seg
  average current: 12.4746